## How CUDA and GPU parallelization works

This notebook works as a short-summary of key-terms and main concepts that are relevant for understanding GPUs and parallelization.

### 1. Summary of key terms

**FLOPS** floating-point operations per second
-> measures the throughput of a GPU. Theoretical peak throughput = (number of cores) × (instructions per core per cycle) × (clock frequency)

**Core**: execution unit capable of performing operations

**Clock cycle**: one “tick” of the processor’s clock

**Fused Multiply-Add (FMA)**: some GPUs can do an addition and multiplication within on clock cycle

**Clock Frequency**: measured in Heartz = cycles per second. 1.5 GHz = 1.5 billion cycles per second

**Memory Bandwidth**: rate at which data can be transferred between GPU memory (VRAM) and GPU compute units. Usually measures in GB/S (gigabytes per second)

**Transistor**: on and off switches that can build logic gates, which can be combined into processors

**Data Caching**: saving a small copy of data close to the processor, such that it doesn't need to be fetched from RAM all the time

**Flow Control**: ensures data flow without bottlenecks or overflow. In GPUs/CPUs it decides when to stall, flush, or forward instructions so the pipeline runs smoothly

**Streaming Multiprocessors (SM)**: building blocks of NVIDIA GPUs that are basically small processors. Has its own cores + memory and is able to run thousands of threads at once.

### 2. GPU architecture & key concepts

##### 2.1 CPU vs. GPU

<figure>
  <img src="./images/gpu-devotes-more-transistors-to-data-processing.png" alt="Architecture comparison CPU vs. GPU" width="500">
  <figcaption><i>Figure 1: Architecture comparison CPU vs. GPU</i></figcaption>
</figure>

In a GPU much more transistors are used for processing data rather than data caching and flow control! This improves parallelism a lot, but comes at the cost of memory latency. In a CPU higher memory latency would be a big problem, as tasks are executed sequentially. But a GPU can do other computations on different threads, as some threads wait for data. This effectively 'hides' the memory latency. At least for tasks that a GPU is designed for.

##### 2.2 Scalable Programming Model

**Main requirement**: Scaling cores or multiple GPUs to achieve faster performance only works, if the software allows for scaled parallelism, meaning that more compute resources allow for more parallel task executions. This requires that the task at hand can be divided into sub-tasks that can be independently executed in any order.


<figure>
  <img src="./images/automatic-scalability.png" alt="Automatic Scalability with more GPUs" width="500">
  <figcaption><i>Figure 2: Automatic Scalability with more GPUs</i></figcaption>
</figure>

### 3. CUDA Programming Model

##### 3.1 Kernels

A kernel is a C++ function in CUDA that is executed N times in parallel by N different CUDA threads. Each threat that executes a kernel is given a thread ID.

Example: A kernel for adding vectors of size Nx1 could be called N times. Therefore, the necessary N addtions would be performed in parallel via N threads with thread IDs ranging from 0 to N-1.

##### 3.2 Thread Hierachy

Thread IDs can be one-, two- or three-dimensional and therefore are structured in a vector, matrix or volume. This provides a natural ways to handle those structures. All thread IDs together are called a thread block. One thread block needs to be processed on one SM and share its limited memory. Therefore, we have a limit of number of threads per thread block. According to the CUDA guide it is 1024 threads per SM.

As the SM needs a thread ID we need to map the index of a thread (3D coordinates of a thread in its block) to the thread ID (flattened 1D number that uniquely identifies the thread in its block).

**1D case**: index of a thread and thread ID are the same<br>
**2D case**: for a thread block (matrix) of size (Dx,Dy) the thread ID is calculated using the following formula: thread ID for entry (x,y) = x + y * Dx. Therefore, we effectively count from 0 to N-1 as written in the section [3.1](#31-kernels).
A matrix of size [3x2] would have the following thread IDs, which is pretty convenient:<br>
[0, 1]<br> 
[2, 3]<br>
[4, 5]<br>

**3D case**: Essentially it is the same, but with 3 instead of two dimensions. We have a thread block (volume) of size (Dx,Dy,Dz). Thread ID for entry (x,y,z) = x + y * Dx + z * Dx * Dy

##### Sources
- CUDA Guide -> https://docs.nvidia.com/cuda/cuda-c-programming-guide/#introduction